# Notebook 9: YOLO Estomas

En este notebook vamos a ver como se entrena la red YOLO desde cero utilizando un dataset de estomas, el fin de este dataset consiste en dadas unas imágenes que nos detecte si hay estomas y donde. 

### 1- Recolección del dataset

Para entrenar YOLO nos hace falta un conjunto de imágenes que formen el dataset. En este caso se va a utilizar un dataset de estomas cedido por la Universidad de Missouri a través de la Universidad de La Rioja. Los estomas son poros o aberturas regulables del tejido epidérmico de las plantas, formados por un par de células especializadas, denominadas células oclusivas o guarda. Al poro en sí, se le denomina ostiolo, que comunica hacia el interior con una cavidad denominada cámara subestomática. Adyacente a cada célula guarda se encuentran generalmente 1 ó 2 células epidérmicas modificadas que reciben el nombre de células subsidiarias o accesorias, siendo las células oclusivas las que controlan la apertura de los estomas como vemos en la siguiente figura. 

<img src="imágenes/estomas.png" style="width:250px;">

El número y comportamiento de los estomas proporciona información clave para medir los niveles de estrés de agua, ratio de producción, y la salud general de la planta. Analizar el número y comportamiento de los estomas puede servir para gestionar mejor los cultivos en agricultura. Sin embargo, contar de manera manual el número de estomas de una imagen es un proceso costoso, por lo que es interesante ser capaces de automatizarlo.
Este dataset cuenta con un total de 468 imágenes etiquetadas por expertos en el campo (las imágenes están anotadas en el formato Pascal VOC utilizando el programa LabelImg).

### 2- Data augmentation
Como tenemos un dataset pequeño, ya que sólo cuenta con 468 imágenes, será necesario aumentarlo aplicándole al conjunto inicial de imágenes una serie de filtros, rotaciones, etc. Para este caso se ha usado una librería desarrollada por la Universidad de La Rioja, llamada CLoDSA. Esta librería permite aplicar un conjunto de transformaciones y a la vez generar de manera automática las anotaciones correspondientes. De este modo, el usuario sólo tiene que anotar el conjunto de imágenes inicial, y la librería se encarga de manera automática de generar las imágenes aumentadas y sus correspondientes anotaciones, evitando la anotación manual de muchas imágenes. Para nuestro caso se han aplicado las siguientes transformaciones: 
* Volteo vertical
* Volteo horizontal 
* Volteo horizontal y vertical
* Rotaciones de 90, 180 y 270 grados
* El filtro de la media
* Ruido gausiano

Aplicamos cada una de estas transformaciones al conjunto inicial, por lo que el dataset inicial se multiplica por 8. Al final de este proceso contamos con un total de 4212 imágenes en el dataset (las 468 originales y las 3744 generadas por CLoDSA).

### 3- Anotación del dataset
Ahora necesitamos que todas las imágenes estén anotadas en el formato YOLO, es decir YOLO necesita que cada imagen tenga un archivo .txt con una  línea por cada objeto que nos interese detectar. El formato de anotación de este dataset, como hemos comentado antes, es en formato Pascal VOC por lo que tenemos que convertir dichas anotaciones al formato YOLO. Para eso usaremos un script, llamado pascal2yolo_1class.py, que convierte las anotaciones de Pascal VOC en formato YOLO. Una vez que ejecutamos este script sobre el dataset ya lo tendremos listo.

<img src="imágenes/anotacionEstomas.png" style="width:500px;">

### 4- Dataset split

En este caso vamos a dividir el dataset en dos conjuntos basándonos en la división 90-10%, por lo que tendremos dos conjuntos, uno de 3744 imágenes (el de entrenamiento) y otro de 468 imágenes (el de test) . 
YOLO necesita un archivo .txt donde tenga listadas las imágenes que va a usar en el entrenamiento y lo mismo en el caso de las de test, estos archivos son test.txt para las imágenes de test y train.txt para las de entrenamiento.

<img src="imágenes/datasetSplit.png" style="width:500px;">


### 5- Entrenamiento

El primer paso antes de empezar a entrenar será cambiar el archivo de configuración cfg/vocEstomas.data para que apunte a las imágenes que hemos descargado y para indicarle que nuestro dataset sólo tiene una clase de objetos.. 

<img src="imágenes/vocEstomas.png" style="width:500px;">

También tenemos que modificar el fichero data/vocEstomas.names para que contenga el nombre de cada una de las clases de nuestro dataset, en este caso solo hay una que es estoma.

<img src="imágenes/vocEstomasNames.png" style="width:500px;">

A parte de este fichero también es necesario que modifiquemos el fichero cfg/yolov3.cfg, indicando que vamos a realizar el entrenamiento:

<img src="imágenes/yolov3Estomas.png" style="width:250px;">

Además de esto, ha sido necesario modificar dentro de este fichero dos parámetros: el parámetro classes (que tiene tres apariciones, y donde debemos indicar el número de clases de nuestro dataset, en este caso 1) y el del parámetro filters (que tiene otras tres apariciones):

<img src="imágenes/vocEstomasClases.png" style="width:250px;">
<img src="imágenes/vocEstomasFilter.png" style="width:250px;">

Con el parámetro filter hay que tener cuidado ya que aparece más de tres veces, sólo hay que cambiar aquellos filter que se encuentren en el bloque *Convolutional*. Al parámetro filter al que asignarle el valor que corresponde a este operación, *filters=(classes + 5)x3*, por lo que en este caso será *filters= (1+5)x3 = 18*.

Hay dos maneras de entrenar la red YOLO: o bien usando unos pesos pre-entrenados o partiendo de cero. En este caso se van a usar unos pesos previamente pre-entrenados llamados *darknet53.conv.74*, estos pesos que usamos provienen del modelo darknet53. Y comenzamos el entrenamiento.

In [0]:
./darknet detector train cfg/vocEstomas.data cfg/yolov3.cfg darknet53.conv.74

El número de épocas que se va a entrenar el modelo es 250000.

### 6- Evaluación
Una vez finalizado el entrenamiento, y antes de comprobar que nos funciona en imágenes reales, vamos a evaluar el modelo construido en el conjunto de test. Para poder ejecutar este test es necesario descargarse una librería [darknet](https://github.com/AlexeyAB/darknet) desde otro sitio que tiene un script que permite ejecutar la siguiente instrucción. Además también hará falta configurar el archivo yolov3.cfg, donde indiquemos que vamos a realizar el test. 

<img src="imágenes/yolov3Estomas2.png" style="width:250px;">
Una vez realizado este paso, ejecutamos la siguiente instrucción.


In [0]:
./darknet detector map cfg/vocEstomas.data cfg/yolov3.cfg backup/yolov3_220000.weights

Este paso debería realizarse sobre los 4 ó 5 últimos pesos que se han generado en el entrenamiento, ya que el último no tiene por que ser el mejor. Esto se hace para evitar usar un modelo que se haya sobreajustado al conjunto de entrenamiento y no generalice bien. 

<img src="imágenes/backup.png" style="width:500px;">

Al acabar, el comando anterior nos mostrará el valor de mAP para cada clase del dataset (en este caso sólo tenemos una), y el valor global de map, lo que nos permitirá clasificarlo como aceptable o no. Veamos los resultados para los últimos pesos obtenidos, en el caso del fichero yolov3_220000:

<img src="imágenes/mapEstomas.png" style="width:500px;">

Para el fichero yolov3_230000:

<img src="imágenes/mapEstomas230000.png" style="width:500px;">

Para el fichero yolov3_240000:

<img src="imágenes/mapEstomas240000.png" style="width:500px;">

Para el fichero yolov3_250000:

<img src="imágenes/mapEstomas250000.png" style="width:500px;">


En este caso todos tienen el mismo valor de mAP, por lo que necesitamos fijarnos en otros parámetros como pueden ser la precisión, el F1-score, el valor del IoU y el recall. 


<img src="imágenes/tablaMap.png" style="width:500px;">


Fijándonos en estos parámetros vemos que el fichero con mejores resultados es el fichero yolov3_250000.  Una vez que ya tenemos el fichero que nos pueda dar los mejores resultado es necesitamos saber si es aceptable o no, en el artículo "Microscope image based fully automated stomata detection and pore measurement method for grapevines" se obtiene una precisión de 91.68% y F1-score de 85% y se considera que es aceptable, mientras que en nuestro caso se ha obtenido una precisión del 98% y un F1-score del 99%, por lo que podemos considerarlo como aceptable además de que su valor mAP es de 90,91%. 


### 7- Predicción
Por último probamos nuestro modelo con imágenes que no se encontraban en el conjunto inicial. Ejecutamos para ello la siguiente instrucción con una imagen.

In [0]:
./darknet detector test cfg/vocEstomas.data cfg/yolov3.cfg backup/yolov3_250000.weights estomas/test/JPEGImages/0_0_1044.1.B3.jpg

<img src="imágenes/predicEstomas.png" style="width:500px;">

Y en la imagen resultado lo vemos:

<img src="imágenes/finalEstomas.png" style="width:500px;">